K_Means from scratch

---


Filip Widera

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
from google.colab import files
uploaded = files.upload()

Saving Iris (1).csv to Iris (1).csv


In [6]:
iris = pd.read_csv("Iris (1).csv") #Load Data
iris.drop('Id',inplace=True,axis=1) #Drop Id column

In [7]:
df = iris

In [8]:
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [10]:
df.describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [11]:
fig = px.pie(iris, 'Species',color_discrete_sequence=['#491D8B','#7D3AC1','#EB548C'],title='Data Distribution',template='plotly')

fig.show()

In [12]:
px.histogram(data_frame = df, x = df['SepalLengthCm'], color = df['Species'])

In [13]:
fig = px.box(data_frame=iris, x='Species',y='SepalLengthCm',color='Species',color_discrete_sequence=['#29066B','#7D3AC1','#EB548C'],orientation='v')
fig.show()

In [14]:
fig = px.histogram(data_frame=iris, x='SepalWidthCm',color='Species',color_discrete_sequence=['#491D8B','#7D3AC1','#EB548C'],nbins=30)
fig.show()

In [15]:
px.box(data_frame = df, x = 'SepalWidthCm', color = 'Species')

In [16]:
X = df.drop(['Species'], axis = 1)

In [66]:
X = X.values


AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [63]:
class Kmeans:

  def __init__(self,K,iterations = 10):
    self.K = K
    self.iterations = iterations

  def initialize_centroid(self, X):
          """
          initialize_centroids(self, X): Initializes the centroids for each cluster by selecting K random points from the dataset.

          Parameters:
          X (numpy.ndarray): data set

          """
          #tworze losowe indexy z puli wszystkich wierszów df
          indx = np.random.permutation(X.shape[0])
          #wybieram k pierwszych losowych indexów
          wybrane = indx[:self.K]
          #tworze macierz centroidów
          self.centroids = X[wybrane]
  def dystans_al(self, x, y):
      """
      Liczy odległość Euklidesową między dwoma wektorami.

      param x: np.array shape (n_features,)
      param y: np.array shape (n_features,)
      return: float
      """
      return np.sqrt(np.sum((x - y) ** 2))

  def euclidean_dist(self, X):
      """
      Liczy macierz odległości między punktami X a centroidami self.centroids.

      param X: np.array shape (n_samples, k_features)
      return: np.array shape (n_samples, n_centroids)
      """
      macierz_odleglosci = []
      for record in X:
          dist = []
          for centroid in self.centroids:
              dist.append(self.dystans_al(record, centroid))  #
          macierz_odleglosci.append(dist)
      return np.array(macierz_odleglosci)

  def assign_points_centroids(self, X):
          """
          Assign each point in the dataset to the nearest centroid.

          Parameters:
          X (numpy.ndarray): dataset to cluster

          Returns:
          numpy.ndarray: array containing the index of the centroid for each point
          """
          macierz_odleglosci = self.euclidean_dist(X)
          Punkty_z_centroidami = np.argmin(macierz_odleglosci, axis = 1)
          return Punkty_z_centroidami
  def compute_mean(self, X, points):
          """
          Compute the mean of the points assigned to each centroid.

          Parameters:
          X (numpy.ndarray): dataset to cluster
          points (numpy.ndarray): array containing the index of the centroid for each point

          Returns:
          numpy.ndarray: array containing the new centroids for each cluster
          """
          cluster = []
          for i in range(self.K):
            cluster.append(np.mean(X[points == i], axis = 0))
          cluster = np.array(cluster)
          return cluster

  def fit(self, X):
        """
        Cluster the dataset using the K-Means algorithm.

        Parameters:
        X (numpy.ndarray): dataset to cluster
        iterations (int): number of iterations to perform (default=10)

        Returns:
        numpy.ndarray: array containing the final centroids for each cluster
        numpy.ndarray: array containing the index of the centroid for each point
        """

        self.initialize_centroid(X)
        for i in range(self.iterations):
          przypisane_punkty = self.assign_points_centroids(X)
          self.centroids = self.compute_mean(X,przypisane_punkty)
        return self.centroids, przypisane_punkty




In [64]:
model = Kmeans(3,1000)
centroids, grupy = model.fit(X)
print(centroids,grupy)

[[5.006      3.418      1.464      0.244     ]
 [5.9016129  2.7483871  4.39354839 1.43387097]
 [6.85       3.07368421 5.74210526 2.07105263]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 2 1 2 2 2 2
 2 2 1 1 2 2 2 2 1 2 1 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2 2 1 2 2 2 1 2 2 2 1 2
 2 1]


In [65]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=X[grupy == 0, 0], y=X[grupy == 0, 1],
    mode='markers',marker_color='#DB4CB2',name='Iris-setosa'
))

fig.add_trace(go.Scatter(
    x=X[grupy == 1, 0], y=X[grupy == 1, 1],
    mode='markers',marker_color='#c9e9f6',name='Iris-versicolour'
))

fig.add_trace(go.Scatter(
    x=X[grupy == 2, 0], y=X[grupy == 2, 1],
    mode='markers',marker_color='#7D3AC1',name='Iris-virginica'
))

fig.add_trace(go.Scatter(
    x=centroids[:, 0], y=centroids[:,1],
    mode='markers',marker_color='#CAC9CD',marker_symbol=4,marker_size=13,name='Centroids'
))
fig.update_layout(template='plotly_dark',width=1000, height=500,)